In [36]:
import pandas as pd
import  numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import iplot
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)
cf.go_offline()

df = pd.read_csv("./odin_data.csv")
countries = df["Country"].unique() 
data_frames=[]

#create a dataframe for avery country 
for country in countries:
    df1=df.loc[df['Country'] == country]
    data_frames.append(df1)

def preprocessing_dataFrame(df1):
    
    yAxis=[0 for x in range(0,26)]
    #print(yAxis)
    df1['yAxis']=yAxis
    cov_score=df1["Coverage subscore"].tolist()
    
    mod_sub_score=[]
    seen={}
    for x in cov_score:
        #print(x)
        if x in seen:
            #print("bin in if")
            key_seen=seen[x]
            key_seen=key_seen-0.7
            seen[x]=key_seen
            mod=x+key_seen
            #print("mod "+str(mod))
            #print("seen[x] "+str(seen[x]))
            mod_sub_score.append(mod)
            #print()
        else:
            #print("bin in else")
            seen[x]=0
            mod_sub_score.append(x)
            #print()
    #print(cov_score)
    #print(mod_sub_score)
    df1['mod_sub_score']=mod_sub_score
    
    size=[10 for x in range(0,26)]
    df1['size']=size
    
    df2=pd.DataFrame(np.repeat(df1.values, 10, axis=0))

    df2.columns=df1.columns
    
    listeY=[]
    x=0.9
    for i in range(0,10):
        listeY.append(x)
        x=x-float(0.1)

    x=-0.9
    for i in range(0,10):
        listeY.append(x)
        x=x+float(0.1)

    #print(listeY)

    yAxis=[]
    for i in range(0,13):
        yAxis=yAxis+listeY


    df2["yAxis"]=yAxis
    
    years=[]
    x=0
    for i in range(0,10):
        years.append(x)
        x=x+1

    years1=[]
    for i in range(0,26):
        years1=years1+years


    df2["Year"]=years1
    
    df2 = df2.rename(columns={'Year': 'Animation'})     
    size=[70 for x in range(0,260)]
    df2["size"]=size
    df2["color"]=["All Categories Score" for x in range(0,260)]
    
    df2["score_text"]=["" for x in range(0,260)]
    
    df2.loc[df2['Data categories'] == "All Categories", "score_text"] = "All Categories Score"
    #Environment subscore
    df2.loc[df2['Data categories'] == "Environment subscore", "score_text"] = "Environmental statistics"
    #Social statistics subscore
    df2.loc[df2['Data categories'] == "Social statistics subscore", "score_text"] = "Social statistics"
    #Economic & financial statistics subscore
    df2.loc[df2['Data categories'] == "Economic & financial statistics subscore", "score_text"] = "Economic & financial statistics"
    #change size
    df2.loc[df2['score_text']!="", "size"] = 120
    #delete text in the end of animation
    df2.loc[abs(df2['yAxis'])<0.15, "score_text"] = ""
    
    economical=["National accounts", "Labor", "Price indexes", "Government finance", "Money & banking", "International trade", "Balance of payments", "Economic & financial statistics subscore" ]
    social=["Population & vital statistics","Education facilities","Education outcomes","Health facilities","Health outcomes","Reproductive health",
           "Food security & nutrition", "Gender statistics", "Crime & justice", "Poverty & income", "Social statistics subscore"]
    enviroment=["Agriculture & Land Use", "Resource use", "Energy", "Pollution", "Built environment", "Environment subscore",]

    
    df2.loc[df2['Data categories'].isin(economical), 'color']="Economic & financial statistics"

    
    df2.loc[df2['Data categories'].isin(social), 'color']='Social statistics'

    
    df2.loc[df2['Data categories'].isin(enviroment), 'color']='Environmental statistics'
    
 
    df2 = df2.rename(columns={'mod_sub_score': 'Data Categories Scores', "color": 'Color Legend'}) 

    return df2


data_frames_prepaired={}
#prepare dataframes for plotly 
for data_frame in data_frames:
    data_frame_prepaired=preprocessing_dataFrame(data_frame)
    df_country=data_frame["Country"].unique()[0]
    data_frames_prepaired[df_country]=data_frame_prepaired
    



In [72]:
def plot(country):

    fig=px.scatter(data_frames_prepaired[country], x='Data Categories Scores', y='yAxis', hover_data=["Data categories"], text="score_text",   
                  animation_frame="Animation", animation_group="Data categories", color='Color Legend',size="size",
                   color_discrete_map={ # replaces default color mapping by value
                    'Social statistics': "RebeccaPurple", "Economic & financial statistics": "red",
                       "All Categories Score":"black", 'Environmental statistics':"#021988"},title=country
                  )


    fig.update_xaxes(tick0=0, dtick=5)
    fig.update_xaxes(tick0=0)
    fig.update_yaxes(range=[-1, 1])
    fig.update_traces(textposition='top center')
    fig.update_traces(textfont=dict(
            color="Black"
        ))
    fig.update_traces(hovertemplate="%{customdata[0]}<extra></extra>")
    for f in fig.frames:
        for i in range(0,len(f.data)):
            f.data[i].update(hovertemplate="%{customdata[0]}<extra></extra>")
    fig.update_traces(showlegend=True)
    fig.update_yaxes(title='yaxis', visible=False, showticklabels=False)


    fig.show()

plot("Cyprus")
